In [ ]:
import nltk
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [ ]:
from nltk.corpus import twitter_samples
postive_tweets=twitter_samples.strings('positive_tweets.json')
negative_tweets=twitter_samples.strings('negative_tweets.json')

In [ ]:
print(len(postive_tweets),len(negative_tweets))

5000 5000


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from nltk.stem import PorterStemmer
import re

In [ ]:
tweet=postive_tweets[0]
tweet=re.sub('(#|@)\w*',"",tweet)
tweet=re.sub('https?:\/\/\S+',"",tweet)
tweet=re.sub('(\?|!)',"",tweet)
tweet=re.sub('\s\d+\s',"",tweet)
tweet=re.sub('(\.|\,)+',"",tweet)
tweet=re.sub('\s+$',"",tweet)
str(tweet.split())

"['for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']"

In [ ]:
#clean text
def clean_text(tweet):
    #clean text
    tweet=re.sub('(#|@)\w*',"",tweet)
    tweet=re.sub('https?:\/\/\S+',"",tweet)
    tweet=re.sub('(\?|!)',"",tweet)
    tweet=re.sub('\s\d+\s',"",tweet)
    tweet=re.sub('(\.|\,)+',"",tweet)
    tweet=re.sub('^\s+',"",tweet)
    tweet=re.sub('\s+$',"",tweet)
    return tweet

In [ ]:
clean_text(postive_tweets[0])

'for being top engaged members in my community this week :)'

In [ ]:
vocab=[]
vocab_size=0

def process_tweets(tweets):

    clean_tweets=[]
    for tweet in tweets:

        #clean tweet
        tweet=clean_text(tweet)
        tweet=tweet.split()
        c_tweet=[word.lower() for word in tweet if word.lower() not in stop_words]

        #steming
        ps=PorterStemmer()
        clean_tweet=[ps.stem(word) for word in c_tweet]


        #count vocab size
        for word in clean_tweet:

            if word not in vocab:
                vocab.append(word)

                global vocab_size
                vocab_size+=1

        clean_tweet=' '.join(clean_tweet)

        clean_tweets.append(clean_tweet)

    return clean_tweets

In [ ]:
positive_tweets=process_tweets(postive_tweets)
negative_tweets=process_tweets(negative_tweets)

In [ ]:
positive_labels=[1]*len(positive_tweets)
negative_labels=[0]*len(negative_tweets)
positive_tweets.extend(negative_tweets)
positive_labels.extend(negative_labels)

tweets=positive_tweets
labels=positive_labels

print(len(tweets),len(labels))

10000 10000


In [ ]:
import random

zip_list=(list(zip(tweets,labels)))
random.shuffle(zip_list)
tweets,labels=zip(*zip_list)

## Text Prepration

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
encoded_tweets=[one_hot(tweet,vocab_size,filters='') for tweet in tweets]

In [ ]:
encoded_tweets[:5]

[[880, 8208],
 [2682, 776, 8208, 9482, 2495, 5304],
 [6735, 6475, 2516, 1455, 8711, 9166, 1455, 5611, 3100, 8208],
 [8748, 4049, 1667, 8786, 9588],
 [10963, 407, 818, 6245, 1771]]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_tweets=pad_sequences(encoded_tweets,maxlen=27,padding='post')
padded_tweets[:5]

array([[  880,  8208,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [ 2682,   776,  8208,  9482,  2495,  5304,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [ 6735,  6475,  2516,  1455,  8711,  9166,  1455,  5611,  3100,
         8208,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [ 8748,  4049,  1667,  8786,  9588,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [10963,   407,   818,  6245,  1771,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
  

In [ ]:
import numpy as np
train_tweets,train_y=padded_tweets[:9000],np.array(labels[:9000])
test_tweets,test_y=padded_tweets[9000:],np.array(labels[9000:])
len(train_tweets),len(test_tweets)

(9000, 1000)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense,Dropout,LSTM,GRU

In [ ]:
vocab_size

11154

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=8,input_length=27))
model.add(LSTM(20,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 8)             89232     
                                                                 
 lstm_2 (LSTM)               (None, 20)                2320      
                                                                 
 dense_6 (Dense)             (None, 10)                210       
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 91,773
Trainable params: 91,773
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(train_tweets,train_y,validation_split=0.1,epochs=5,batch_size=64)

Epoch 1/5
127/127 [==============================] - 5s 20ms/step - loss: 0.6840 - accuracy: 0.5226 - val_loss: 0.4956 - val_accuracy: 0.9389
Epoch 2/5
127/127 [==============================] - 2s 17ms/step - loss: 0.6888 - accuracy: 0.8752 - val_loss: 0.4638 - val_accuracy: 0.9600
Epoch 3/5
127/127 [==============================] - 3s 25ms/step - loss: 0.3589 - accuracy: 0.9436 - val_loss: 0.1241 - val_accuracy: 0.9700
Epoch 4/5
127/127 [==============================] - 2s 17ms/step - loss: 0.2452 - accuracy: 0.9526 - val_loss: 0.0684 - val_accuracy: 0.9767
Epoch 5/5
127/127 [==============================] - 2s 16ms/step - loss: 0.1982 - accuracy: 0.9633 - val_loss: 0.0609 - val_accuracy: 0.9789


In [ ]:
model.evaluate(train_tweets,train_y)

282/282 [==============================] - 2s 6ms/step - loss: 0.0206 - accuracy: 0.9944


[0.020566951483488083, 0.9944444298744202]

In [ ]:
model.evaluate(test_tweets,test_y)

32/32 [==============================] - 0s 5ms/step - loss: 0.1169 - accuracy: 0.9670


[0.11693668365478516, 0.9670000076293945]